In [1]:
from typing import TypedDict, List, Literal
from pydantic import BaseModel, Field

class RAG(BaseModel) : 

    answer :str = Field(description="The answer to the question")
    source_documents : List[str] = Field(description="The source documents used to generate the answer")


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    number_of_document_tries: int
    documents: List[str]
    upload_status: str
    source_type: str

In [3]:

from langchain_huggingface import HuggingFaceEmbeddings


embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/mahdi/projects/Aegis/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:

from langchain_astradb import AstraDBVectorStore
import os

vectorstore = AstraDBVectorStore(
                embedding=embedding,
                api_endpoint=os.getenv('ASTRA_DB_API_ENDPOINT'),
                token=os.getenv('ASTRA_DB_APPLICATION_TOKEN'),
                namespace= None,
                collection_name="astra_vector_langchain", 
            )

retriever=vectorstore.as_retriever()

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq 
llm = ChatGroq(model="moonshotai/kimi-k2-instruct-0905")
system = """You are a grader assessing whether an LLM generation answers a user question.

            You must respond with ONLY a JSON object in this exact format:
            {{"binary_score": "yes"}} or {{"binary_score": "no"}}

            Rules:
            - "yes" means the answer addresses and answers the user's question
            - "no" means the answer does not address the user's question
            - Respond with ONLY the JSON object, nothing else"""
            
answer_prompt = ChatPromptTemplate.from_messages([
                ("system", system),
                ("human", "User question: \n\n {question} \n\n LLM generation: {generation}\n\nResponse (JSON only):"),
            ])

from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
            
answer_grader = answer_prompt |llm | json_parser

In [6]:
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts.

            You must respond with ONLY a JSON object in this exact format:
            {{"binary_score": "yes"}} or {{"binary_score": "no"}}

            Rules:
            - "yes" means the answer is grounded in and supported by the provided facts
            - "no" means the answer contains information not supported by the facts
            - Respond with ONLY the JSON object, nothing else"""
            
hallucination_prompt = ChatPromptTemplate.from_messages([
                ("system", system),
                ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}\n\nResponse (JSON only):"),
            ])

            # Use JSON mode instead of structured output for better reliability
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

hallucination_grader = hallucination_prompt | llm | json_parser


In [7]:
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
            for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system),
                    (
                        "human",
                        "Here is the initial question: \n\n {question} \n Formulate an improved question.",
                    ),
                ]
            )

question_rewriter = re_write_prompt | llm | StrOutputParser()


In [8]:
system = """You are an expert at routing user questions to either a vectorstore or web search.

            You must respond with ONLY a JSON object in this exact format:
            {{"datasource": "vectorstore"}} or {{"datasource": "web_search"}}

            The vectorstore contains documents about: AI agents, prompt engineering, and adversarial attacks.
            
            Rules:
            - Use "vectorstore" for questions about: AI agents, prompt engineering, adversarial attacks, LLMs, machine learning concepts
            - Use "vectorstore" for simple greetings, casual conversations, or non-specific questions (like "hi", "hello", "how are you")
            - Use "web_search" ONLY for questions requiring current/recent information, news, weather, or specific factual queries not covered by the vectorstore
            
            Default to "vectorstore" unless you're certain the question needs current web information.
            Respond with ONLY the JSON object, nothing else."""
            
route_prompt = ChatPromptTemplate.from_messages([
                ("system", system),
                ("human", "{question}\n\nResponse (JSON only):"),
            ])

from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

question_router = route_prompt | llm | json_parser


In [9]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt") 

                # Fallback to custom prompt
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
                    ("system", """You are a helpful AI assistant. Use the provided context to answer questions accurately.
                    
                    If the question is a simple greeting or casual conversation (like "hi", "hello", "how are you"), 
                    respond naturally and friendly without requiring specific context.
                    
                    For knowledge questions, use the provided context to give accurate, helpful answers.
                    If you don't have enough context to answer a question, say so clearly.
                    
                    Context: {context}"""),
                    ("human", "{question}")
                ])
            
rag_chain = prompt | llm | StrOutputParser() 


In [23]:
system = """You are a grader assessing relevance of a retrieved document to a user question.

            You must respond with ONLY a JSON object in this exact format:
            {{"binary_score": "yes"}} or {{"binary_score": "no"}}

            Rules:
            - If the document contains keywords or semantic meaning related to the user question, grade it as "yes"
            - It does not need to be a stringent test. The goal is to filter out completely irrelevant retrievals
            - "yes" means the document is relevant to the question
            - "no" means the document is not relevant to the question
            - Respond with ONLY the JSON object, nothing else"""
            
grade_prompt = ChatPromptTemplate.from_messages([
                ("system", system),
                ("human", "Retrieved document: \n\n {document} \n\n User question: {question}\n\nResponse (JSON only):"),
            ])
            
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
            
retrieval_grader = grade_prompt | llm | json_parser

In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(
    api_key=os.getenv("TAVILY_API_KEY"),
    k=3,
)

/tmp/ipykernel_165415/1277798333.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


In [24]:
from db_test import test_astra_connection
from langchain_astradb import AstraDBVectorStore

import os
from dotenv import load_dotenv
from langchain_core.documents import Document
load_dotenv()

class upload_generated_answers:

    def __init__ (self, documents, answer): 
        self.source_documents = documents
        self.answer = answer
        self.embedder = embedding  # Use the global embedding variable
        self.vectorstore = None


    def upload_answer(self):
        try: 
            api_endpoint = os.getenv('ASTRA_DB_API_ENDPOINT')
            token = os.getenv('ASTRA_DB_APPLICATION_TOKEN')

            testing_db = test_astra_connection()
            if testing_db == True : 
                print("Connected to Astra DB successfully!")
                self.vectorstore = AstraDBVectorStore(
                embedding=self.embedder,
                api_endpoint=api_endpoint,
                token=token,
                collection_name="astra_vector_langchain",
            )
                # Extract text content from documents for metadata (JSON serializable)
                if hasattr(self.source_documents, 'page_content'):
                    # Single document
                    source_text = [self.source_documents.page_content]
                elif isinstance(self.source_documents, list):
                    # List of documents
                    source_text = [doc.page_content if hasattr(doc, 'page_content') else str(doc) for doc in self.source_documents]
                else:
                    # Fallback
                    source_text = [str(self.source_documents)]
                
                Document_to_upload = Document(
                    page_content=self.answer,
                    metadata={"source_documents": source_text, "producer" : "Generated_Web_Search_Answer"}
                )

            print("Uploading answer and source documents to Astra DB...")
            self.vectorstore.add_documents([Document_to_upload])

            return "Upload successful"
        except Exception as e: 
            raise ValueError(f"Error occurred with exception : {e}")

In [17]:

from langgraph.graph import StateGraph, START, END

from langgraph.checkpoint.memory import MemorySaver

In [25]:
def retrieve(state: GraphState):
        """
        Retrieve documents

        Args:
            state (dict): The current graph state

        Returns:
            state (dict): New key added to state, documents, that contains retrieved documents
        """
        print("---RETRIEVE---")
        question = state["question"]
        attempts = state.get("number_of_document_tries", 0)
        upload_status = state.get("upload_status", "False")
        
        # Retrieval
        documents = retriever.invoke(question)
        return {
            "documents": documents, 
            "question": question, 
            "number_of_document_tries": attempts, 
            "upload_status": upload_status,
            "source_type": "vectorstore"
        }
    

def generate(state: GraphState): 
        """
        Generate answer

        Args:
            state (dict): The current graph state

        Returns:
            state (dict): New key added to state, generation, that contains LLM generation
        """
        print("---GENERATE---")
        question = state["question"]
        documents = state["documents"]
        attempts = state.get("number_of_document_tries", 0)
        upload_status = state.get("upload_status", "False")
        source_type = state.get("source_type", "unknown")

        # RAG generation
        generation = rag_chain.invoke({"context": documents, "question": question})
        return {
            "documents": documents, 
            "question": question, 
            "generation": generation, 
            "number_of_document_tries": attempts, 
            "upload_status": upload_status,
            "source_type": source_type
        }
    

def grade_documents(state: GraphState):
        """
        Determines whether the retrieved documents are relevant to the question.

        Args:
            state (dict): The current graph state

        Returns:
            state (dict): Updates documents key with only filtered relevant documents
        """

        print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
        question = state["question"]
        documents = state["documents"]
        attempts = state.get("number_of_document_tries", 0)
        upload_status = state.get("upload_status", "False")
        source_type = state.get("source_type", "unknown")

        # Score each doc
        filtered_docs = []
        for d in documents:
            score = retrieval_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            # Handle both dict and Pydantic model responses
            grade = score.binary_score if hasattr(score, 'binary_score') else score.get('binary_score', 'no')
            if grade == "yes":
                print("---GRADE: DOCUMENT RELEVANT---")
                filtered_docs.append(d)
            else:
                print("---GRADE: DOCUMENT NOT RELEVANT---")
                continue
        return {
            "documents": filtered_docs, 
            "question": question, 
            "number_of_document_tries": attempts, 
            "upload_status": upload_status,
            "source_type": source_type
        }
    


def transform_query(state: GraphState):
        """
        Transform the query to produce a better question.

        Args:
            state (dict): The current graph state

        Returns:
            state (dict): Updates question key with a re-phrased question
        """

        print("---TRANSFORM QUERY---")
        question = state["question"]
        documents = state["documents"]
        current_attempts = state.get("number_of_document_tries", 0)
        upload_status = state.get("upload_status", "False")
        source_type = state.get("source_type", "unknown")

        # Re-write question
        better_question = question_rewriter.invoke({"question": question})
        
        # Increment the number of attempts
        new_attempts = current_attempts + 1
        print(f"---INCREMENTING ATTEMPTS: {new_attempts}---")
        
        return {
            "documents": documents, 
            "question": better_question, 
            "number_of_document_tries": new_attempts, 
            "upload_status": upload_status,
            "source_type": source_type
        }
    


def web_search(state: GraphState):
        """
        Web search based on the re-phrased question.

        Args:
            state (dict): The current graph state

        Returns:
            state (dict): Updates documents key with appended web results
        """

        print("---WEB SEARCH---")
        question = state["question"]
        attempts = state.get("number_of_document_tries", 0)
        upload_status = state.get("upload_status", "False")

        # Web search
        docs = web_search_tool.invoke({"query": question})
        web_results = "\n".join([d["content"] for d in docs])
        web_results = Document(page_content=web_results)

        return {
            "documents": web_results, 
            "question": question, 
            "number_of_document_tries": attempts, 
            "upload_status": upload_status,
            "source_type": "websearch"
        }
    


    ### edges 

def route_question(state: GraphState):
        """
        Route question to web search or RAG.

        Args:
            state (dict): The current graph state

        Returns:
            str: Next node to call
        """

        print("---ROUTE QUESTION---")
        question = state["question"]
        source = question_router.invoke({"question": question})
        # Handle both dict and Pydantic model responses
        datasource = source.datasource if hasattr(source, 'datasource') else source.get('datasource', 'vectorstore')
        
        if datasource == "web_search":
            print("---ROUTE QUESTION TO WEB SEARCH---")
            return "web_search"
        elif datasource == "vectorstore":
            print("---ROUTE QUESTION TO RAG---")
            return "vectorstore"
        
        
def grade_generation_v_documents_and_question(state:GraphState ):
        """
        Determines whether the generation is grounded in the document and answers question.
        Only routes to human-in-the-loop for web search results.

        Args:
            state (dict): The current graph state

        Returns:
            str: Decision for next node to call
        """

        print("---CHECK HALLUCINATIONS---")
        question = state["question"]
        documents = state["documents"]  
        generation = state["generation"]
        source_type = state.get("source_type", "unknown")

        score = hallucination_grader.invoke(
            {"documents": documents, "generation": generation}
        )
        # Handle both dict and Pydantic model responses
        grade = score.binary_score if hasattr(score, 'binary_score') else score.get('binary_score', 'no')

        # Check hallucination
        if grade == "yes":
            print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
            # Check question-answering
            print("---GRADE GENERATION vs QUESTION---")
            score = answer_grader.invoke({"question": question, "generation": generation})
            # Handle both dict and Pydantic model responses
            grade = score.binary_score if hasattr(score, 'binary_score') else score.get('binary_score', 'no')
            if grade == "yes":
                print("---DECISION: GENERATION ADDRESSES QUESTION---")
                # Only allow human-in-the-loop for web search results
                if source_type == "websearch":
                    print("---WEB SEARCH RESULT: ROUTE TO HUMAN DECISION---")
                    return "useful_websearch"
                else:
                    print("---VECTOR STORE RESULT: END PROCESS---")
                    return "useful_vectorstore"
            else:
                print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                return "not useful"
        else:
            print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
            return "not supported"
        

def route_question_after_attempts(state: GraphState):
        """
        Route based on document relevance and attempt count.
        
        Args:
            state (dict): The current graph state

        Returns:
            str: Next node to call
        """

        print("---ASSESS GRADED DOCUMENTS---")
        filtered_documents = state["documents"]
        attempts = state.get("number_of_document_tries", 0)

        if not filtered_documents:
            # No relevant documents found
            print("---NO RELEVANT DOCUMENTS FOUND---")
            print(f"---ATTEMPT NUMBER: {attempts}---")
            
            if attempts >= 3:
                print("---ROUTE TO WEB SEARCH AFTER 3 ATTEMPTS---")
                return "web_search"
            else:
                print("---ROUTE TO TRANSFORM QUERY (CONTINUE TRYING)---")
                return "transform_query"
        else:
            # We have relevant documents, so generate answer
            print("---DECISION: GENERATE---")
            return "generate"
        

def human_in_the_loop(state: GraphState):
        """
        Human-in-the-loop intervention for deciding whether to upload web search results.
        
        This node will be interrupted by the graph, allowing human decision making.
        The human can set upload_status in the state to control the next action.

        Args:
            state (dict): The current graph state
            
        Returns:
            state (dict): State with human decision for upload
        """
        print("---HUMAN IN THE LOOP INTERVENTION REQUIRED---")
        print("---WAITING FOR HUMAN DECISION ON UPLOADING WEB SEARCH RESULT---")
        
        question = state["question"]
        generation = state["generation"]
        documents = state["documents"]
        attempts = state.get("number_of_document_tries", 0)
        source_type = state.get("source_type", "unknown")
        
        # The upload_status will be set by human intervention
        # Default to "False" if not set by human
        upload_status = state.get("upload_status", "False")
        
        print(f"Question: {question}")
        print(f"Generated Answer: {generation}")
        print(f"Source Type: {source_type}")
        print(f"Upload Status: {upload_status}")
        
        return {
            "question": question,
            "generation": generation,
            "documents": documents,
            "number_of_document_tries": attempts,
            "upload_status": upload_status,
            "source_type": source_type
        }
        

def decide_to_upload(state: GraphState):
        """
        Decide whether to upload the generated answer to vector store based on human decision.
        
        Args:
            state (dict): The current graph state
            
        Returns:
            str: "yes" to upload, "no" to skip
        """
        print("---DECIDE TO UPLOAD---")
        upload_status = state.get("upload_status", "False")
        
        if upload_status.lower() in ["true", "yes", "1"]:
            print("---DECISION: UPLOAD TO VECTOR STORE---")
            return "yes"
        else:
            print("---DECISION: DO NOT UPLOAD---")
            return "no"
        

def send_answer_vectorstore(state: GraphState):
        """
        Send answer from websearch to vectorstore

        Args:
            state (dict): The current graph state
            
        Returns:
            state (dict): Updated state after upload
        """

        print("---SENDING ANSWER FROM WEBSEARCH TO VECTORSTORE---")
        question = state["question"]
        answer = state["generation"]
        documents = state["documents"]
        attempts = state.get("number_of_document_tries", 0)
        source_type = state.get("source_type", "unknown")

        print("---PREPARING TO UPLOAD GENERATED ANSWER AND SOURCE DOCUMENTS TO ASTRA DB---")
        print(f"---SOURCE TYPE: {source_type}---")

        try:
            uploader = upload_generated_answers(documents, answer)
            upload_result = uploader.upload_answer()
            print("---UPLOAD SUCCESSFUL---")
            
            return {
                "question": question,
                "generation": answer,
                "documents": documents,
                "number_of_document_tries": attempts,
                "upload_status": "completed",
                "source_type": source_type
            }
        except Exception as e:
            print(f"---UPLOAD FAILED: {e}---")
            return {
                "question": question,
                "generation": answer,
                "documents": documents,
                "number_of_document_tries": attempts,
                "upload_status": "failed",
                "source_type": source_type
            }

In [27]:
# Create a new graph instance
graph = StateGraph(GraphState)

In [28]:
graph.add_node("web_search", web_search) 
graph.add_node("retrieve", retrieve)
graph.add_node("grade_documents", grade_documents)
graph.add_node("generate", generate)
graph.add_node("transform_query", transform_query)
graph.add_node("human_in_the_loop", human_in_the_loop)
graph.add_node("send_answer_vectorstore", send_answer_vectorstore)

In [29]:
graph.add_conditional_edges(
            START, 
            route_question, 
            {
                "web_search": "web_search",
                "vectorstore": "retrieve"
            },
        )
        
graph.add_edge("web_search", "generate")
graph.add_edge("retrieve", "grade_documents")

graph.add_conditional_edges(
    "grade_documents",
    route_question_after_attempts,
    {
                "generate": "generate",
                "web_search": "web_search",
                "transform_query": "transform_query"
            },
        )
        
graph.add_edge("transform_query", "retrieve")

graph.add_conditional_edges(
            "generate",
            grade_generation_v_documents_and_question,
            {
                "not supported": END,  # End if hallucinated - avoid infinite loops
                "useful_websearch": "human_in_the_loop",  # Go to human decision for web search uploads
                "useful_vectorstore": END,  # End directly for vector store results
                "not useful": "transform_query"  # Only retry if answer doesn't address question
            },
        )   

        # Human decides whether to upload the web search result to vector store
graph.add_conditional_edges(
            "human_in_the_loop",
            decide_to_upload,
            {
                "yes": "send_answer_vectorstore",
                "no": END
            },
        )
        
graph.add_edge("send_answer_vectorstore", END)

# Compile the graph
memory_saver = MemorySaver()
compiled_graph = graph.compile(
            interrupt_before=["human_in_the_loop"],
            checkpointer=memory_saver
        )

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [66]:
# Test the compiled graph
print("Graph compiled successfully!")
print(f"Graph nodes: {list(compiled_graph.get_graph().nodes)}")
print(f"Graph has {len(compiled_graph.get_graph().nodes)} nodes")

# Test with a simple question (this will require human intervention if it goes to web search)
test_config = {"configurable": {"thread_id": "test_thread"}}
print("\nReady to process questions!")
print("Example usage:")
print('result = compiled_graph.invoke({"question": "Hello, how are you?"}, config=test_config)')

Graph compiled successfully!
Graph nodes: ['__start__', 'web_search', 'retrieve', 'grade_documents', 'generate', 'transform_query', 'human_in_the_loop', 'send_answer_vectorstore', '__end__']
Graph has 9 nodes

Ready to process questions!
Example usage:
result = compiled_graph.invoke({"question": "Hello, how are you?"}, config=test_config)


In [72]:
# Update the state and continue execution
compiled_graph.update_state(config, {
    "upload_status": "False"  
})

# Continue the graph execution after updating state
result = compiled_graph.invoke(None, config)sult = compiled_graph.invoke({"question":"What is prompt engineering?" , "number_of_document_tries":0 }, config=config)

---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
---WEB SEARCH RESULT: ROUTE TO HUMAN DECISION---
---DECISION: GENERATION ADDRESSES QUESTION---
---WEB SEARCH RESULT: ROUTE TO HUMAN DECISION---


---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
---WEB SEARCH RESULT: ROUTE TO HUMAN DECISION---
---DECISION: GENERATION ADDRESSES QUESTION---
---WEB SEARCH RESULT: ROUTE TO HUMAN DECISION---


{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09e02b-ccdc-6734-8010-d07d845f2dc4'}}

In [86]:
resumed_result = compiled_graph.invoke(None, config)
resumed_result

---HUMAN IN THE LOOP INTERVENTION REQUIRED---
---WAITING FOR HUMAN DECISION ON UPLOADING WEB SEARCH RESULT---
Question: what is jordan
Generated Answer: Jordan can refer to two very different things:

1. **The country**: Jordan is a country in the Middle East, slightly smaller than Portugal. It borders Syria to the north, Iraq to the east, Saudi Arabia to the south, and Israel and the West Bank to the west. It became an independent kingdom in 1946 after being part of the Ottoman Empire and later a British mandate. Jordan is known for its ancient history, including the biblical kingdoms of Moab, Gilead, and Edom, and the famous archaeological site of Petra.

2. **Jordan Peterson**: A Canadian psychologist, author, and media commentator (born 1962). He gained widespread attention for his views on cultural and political issues and is known for his books like "12 Rules for Life" and "Beyond Order".
Source Type: websearch
Upload Status: yes
---DECIDE TO UPLOAD---
---DECISION: UPLOAD TO VECT

---HUMAN IN THE LOOP INTERVENTION REQUIRED---
---WAITING FOR HUMAN DECISION ON UPLOADING WEB SEARCH RESULT---
Question: what is jordan
Generated Answer: Jordan can refer to two very different things:

1. **The country**: Jordan is a country in the Middle East, slightly smaller than Portugal. It borders Syria to the north, Iraq to the east, Saudi Arabia to the south, and Israel and the West Bank to the west. It became an independent kingdom in 1946 after being part of the Ottoman Empire and later a British mandate. Jordan is known for its ancient history, including the biblical kingdoms of Moab, Gilead, and Edom, and the famous archaeological site of Petra.

2. **Jordan Peterson**: A Canadian psychologist, author, and media commentator (born 1962). He gained widespread attention for his views on cultural and political issues and is known for his books like "12 Rules for Life" and "Beyond Order".
Source Type: websearch
Upload Status: yes
---DECIDE TO UPLOAD---
---DECISION: UPLOAD TO VECT

{'question': 'what is jordan',
 'generation': 'Jordan can refer to two very different things:\n\n1. **The country**: Jordan is a country in the Middle East, slightly smaller than Portugal. It borders Syria to the north, Iraq to the east, Saudi Arabia to the south, and Israel and the West Bank to the west. It became an independent kingdom in 1946 after being part of the Ottoman Empire and later a British mandate. Jordan is known for its ancient history, including the biblical kingdoms of Moab, Gilead, and Edom, and the famous archaeological site of Petra.\n\n2. **Jordan Peterson**: A Canadian psychologist, author, and media commentator (born 1962). He gained widespread attention for his views on cultural and political issues and is known for his books like "12 Rules for Life" and "Beyond Order".',
 'number_of_document_tries': 0,
 'documents': Document(metadata={}, page_content='Jordan is a young state that occupies an ancient land, one that bears the traces of many civilizations. Separa

In [94]:
next_nodes = state.next
next_nodes

('human_in_the_loop',)